In [6]:
import requests                  # 웹 페이지의 HTML을 가져오는 모듈
from bs4 import BeautifulSoup    # HTML을 파싱하는 모듈
import time
import json
import urllib.request
import pymysql
import os

WorldID = {
    "리부트2" : 46,
    "리부트" : 45,
    "오로라" : 44,
    "레드" : 43,
    "이노시스" : 29,
    "유니온" : 10,
    "스카니아" : 0,
    "루나" : 3,
    "제니스" : 4,
    "크로아" : 5,
    "베라" : 1,
    "엘리시움" : 16,
    "아케인" : 50,
    "노바" : 51,
    "버닝" : 49,
    "버닝2" : 48,
    "버닝3" : 52
}

conn = pymysql.connect(
    user='root',
    passwd='dhkswjsvhrltjsdjs!',
    host='127.0.0.1',
    db='guild_management',
    charset='utf8'
)
cursor = conn.cursor(pymysql.cursors.DictCursor)

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
def find_Dojang (name) :
    dojang_floor = 'NON'
    dojang_time = 'NON'
   
    dojang_cur_url = 'https://maple.gg/u/' + name
    
    print(dojang_cur_url)
    
    dojang_cur_res = requests.get(dojang_cur_url)
    dojang_cur_s = BeautifulSoup(dojang_cur_res.content, 'html.parser')

    if dojang_cur_s.find('h1', {'class' : 'user-summary-floor'}) :
        dojang_best_info = dojang_cur_s.find('h1', {'class' : 'user-summary-floor'}).text.replace(' ','').replace('\n','')
        if int(dojang_best_info.replace('층','')) < 10 :
            dojang_best_info = '0' + dojang_best_info
    else:
        dojang_best_info = '00층'

    if(dojang_cur_s.find('tbody')) :
        dojang_latest_info = dojang_cur_s.find('tbody').find('tr').find('h5').text
        if int(dojang_latest_info.replace('층','')) < 10 :
            dojang_latest_info = '0' + dojang_latest_info
    else :
        dojang_latest_info = '00층'
    
    try:
        dojang_class_info = dojang_cur_s.find_all('li', {'class' : 'user-summary-item'})[1].text
    
        return [dojang_best_info, dojang_latest_info, dojang_class_info]
    except:
        return ["", "", ""]
    



def make_guild_data_mk2 (GuildName, World) :
    guild_select_url = "https://maplestory.nexon.com/Ranking/World/Guild?t=1&n=" + GuildName;
    SELECTED_GID = -1

    time.sleep(0.5)
    # 웹 페이지를 가져온 뒤 BeautifulSoup 객체로 만듦
    response = requests.get(guild_select_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    gid = []
    wid = []

    table = soup.find('tbody')

    if table :
        for tr in table.find_all('tr'):      # 모든 <tr> 태그를 찾아서 반복(각 지점의 데이터를 가져옴)
            tds = list(tr.find_all('td'))    # 모든 <td> 태그를 찾아서 리스트로 만듦

            a = tr.find('a')
            gid.append(a.get('href').split('?')[1].split('&')[0].split('=')[1])
            wid.append(a.get('href').split('?')[1].split('&')[1].split('=')[1])

        i = 0
        
        isExist = False
        
        for selected in wid :
            if selected == World:
                SELECTED_GID = gid[i]
                isExist = True
            
            i = i + 1   

        if isExist == False :
            return "No data"
            
        page_num = 1
        data = []

        while True :

            guild_url = 'https://maplestory.nexon.com/Common/Guild?gid=' + SELECTED_GID + '&wid=' + World + '&orderby=1&page=' + str(page_num)

            res = requests.get(guild_url)
            time.sleep(0.5)
            s = BeautifulSoup(res.content, 'html.parser')

            table = s.find('table')

            if table :
                1
            else :
                break

            for tr in table.find_all('tr'):
                tds = list(tr.find_all('td'))

                for td in tds:
                    if td.find('span', {'class' : 'char_img'}):
                        col1 = tds[0].text.replace(' ', '').replace('\n', '').replace('\r', '') # 직위
                        col2 = td.find('span', {'class' : 'char_img'}).find('img').get('src')   # 케릭터 그림
                        col3 = tds[2].text.replace(' ', '').replace('\n', '').replace('\r', '') # 레벨
                        col4 = tds[3].text.replace(' ', '').replace('\n', '').replace('\r', '') # 경험치
                        col5 = tds[4].text.replace(' ', '').replace('\n', '').replace('\r', '') # 인기도

                        col21 = tds[1].find('a').text # 닉네임
                        col22 = tds[1].find('dd').text # 직업 (대분류)
                        
                        
                        Dojang_log = find_Dojang(str(col21))
                        
                        
                        col = {'position' : col1,
                               'img_src' : col2,
                               'name' : col21,
                               'class' : Dojang_log[2],
                               'Lv' : col3,
                               'exp' : col4,
                               'popularity' : col5,
                               'dojang_best_info' : Dojang_log[0],
                               'dojang_latest_info' : Dojang_log[1]}
                        
                        data.append(col)

            page_num+=1

        print('success')

    else :
        return 'No Data'
        
    return data;

GuildName = "Lune"
World =  str(WorldID['이노시스'])

print(GuildName, World)


def weekly_prestige (GuildName, World) :
    guild_select_url = "https://maplestory.nexon.com/Ranking/World/Guild?t=1&n=" + GuildName;
    SELECTED_GID = -1
    
    time.sleep(0.5)
    # 웹 페이지를 가져온 뒤 BeautifulSoup 객체로 만듦
    response = requests.get(guild_select_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    gid = []
    wid = []
    prestiges = []

    table = soup.find('tbody')

    if table :
        for tr in table.find_all('tr'):      # 모든 <tr> 태그를 찾아서 반복(각 지점의 데이터를 가져옴)
            tds = list(tr.find_all('td'))    # 모든 <td> 태그를 찾아서 리스트로 만듦

            a = tr.find('a')
            gid.append(a.get('href').split('?')[1].split('&')[0].split('=')[1])
            wid.append(a.get('href').split('?')[1].split('&')[1].split('=')[1])
            prestiges.append(tds[4].text)

        i = 0
        
        isExist = False
        
        for selected in wid :
            if selected == World:
                return prestiges[i]
                isExist = True
            
            i = i + 1   

        if isExist == False :
            return "No data"
        
    else : 
        return "No data"

def weekly_flag (GuildName, World) :
    guild_select_url = "https://maplestory.nexon.com/Ranking/World/Guild?t=0&n=" + GuildName;
    SELECTED_GID = -1

    time.sleep(0.5)
    # 웹 페이지를 가져온 뒤 BeautifulSoup 객체로 만듦
    response = requests.get(guild_select_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    gid = []
    wid = []
    prestiges = []

    table = soup.find('tbody')

    if table :
        for tr in table.find_all('tr'):      # 모든 <tr> 태그를 찾아서 반복(각 지점의 데이터를 가져옴)
            tds = list(tr.find_all('td'))    # 모든 <td> 태그를 찾아서 리스트로 만듦

            a = tr.find('a')
            gid.append(a.get('href').split('?')[1].split('&')[0].split('=')[1])
            wid.append(a.get('href').split('?')[1].split('&')[1].split('=')[1])
            prestiges.append(tds[4].text)

        i = 0
        
        isExist = False
        
        for selected in wid :
            if selected == World:
                return prestiges[i]
                isExist = True
            
            i = i + 1   

        if isExist == False :
            return "No data"
        
    else : 
        return "No data"

def weekly_rade (GuildName, World) :
    guild_select_url = "https://maplestory.nexon.com/Ranking/World/Guild?t=2&n=" + GuildName;
    SELECTED_GID = -1

    time.sleep(0.5)
    # 웹 페이지를 가져온 뒤 BeautifulSoup 객체로 만듦
    response = requests.get(guild_select_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    gid = []
    wid = []
    prestiges = []

    table = soup.find('tbody')

    if table :
        for tr in table.find_all('tr'):      # 모든 <tr> 태그를 찾아서 반복(각 지점의 데이터를 가져옴)
            tds = list(tr.find_all('td'))    # 모든 <td> 태그를 찾아서 리스트로 만듦

            a = tr.find('a')
            gid.append(a.get('href').split('?')[1].split('&')[0].split('=')[1])
            wid.append(a.get('href').split('?')[1].split('&')[1].split('=')[1])
            prestiges.append(tds[4].text)

        i = 0
        
        isExist = False
        
        for selected in wid :
            if selected == World:
                return prestiges[i]
                isExist = True
            
            i = i + 1   

        if isExist == False :
            return "No data"
    else : 
        return "No data"

        
def data_input(World, GuildName, flag, rade, data): #데이터 입력
    if rade == 'No data':
        rade = 0
    else :
        rade = rade.replace(',','')
    if flag == 'No data':
        flag = 0
    else :
        flag = flag.replace(',','')

    sql = "INSERT INTO guild_info (input_date, world, name, flag, rade) VALUES ( CURDATE(), '" + World + "', '" + GuildName + "', '" + (flag) + "', '" + (rade) + "')"
    
    cursor.execute(sql)
    conn.commit()
    
    for datas in data:
    
        sql = ( "INSERT INTO char_info (input_date, world, gname, position, name, class, lv, exp, popularity, dojang_best_info, dojang_latest_info) "
               "VALUES ( CURDATE(), '" + World + "', '" + GuildName + "', '" + datas['position'] + "', '" + datas['name'] + "', '" + datas['class'] + "', '" + datas['Lv'] + "', '" + datas['exp'] + "', '" + datas['popularity'] + "', '" + datas['dojang_best_info'] + "', '" + datas['dojang_latest_info'] + "' )" )
        cursor.execute(sql)
        conn.commit()
        url = datas['img_src']
        createFolder('../images/' + World + '/' + GuildName)
        urllib.request.urlretrieve(url, '../images/' + World + '/' + GuildName + '/' + datas['name'] + ".png")
        time.sleep(0.5)
    


GuildName = "Lune"
World =  str(WorldID['이노시스'])


data =  make_guild_data_mk2(GuildName, World)
prestige = weekly_prestige(GuildName, World)
flag = weekly_flag(GuildName, World)
rade = weekly_rade(GuildName, World)


Lune 29
https://maple.gg/u/라쇼님
https://maple.gg/u/R정로
https://maple.gg/u/쁘나델
https://maple.gg/u/레프폴
https://maple.gg/u/프리s천사
https://maple.gg/u/성녀의이야기
https://maple.gg/u/해동카이저
https://maple.gg/u/Diggin
https://maple.gg/u/허렵
https://maple.gg/u/나쁜암떼시
https://maple.gg/u/단풍잎경찰관
https://maple.gg/u/키우는중임돠
https://maple.gg/u/박주성9
https://maple.gg/u/에이스콜리
https://maple.gg/u/시묫
https://maple.gg/u/호우넬
https://maple.gg/u/않대
https://maple.gg/u/프룬화
https://maple.gg/u/영블리쓰
https://maple.gg/u/베에에에
https://maple.gg/u/눈까리파삘라
https://maple.gg/u/딸기맛태엽
https://maple.gg/u/민셔림
https://maple.gg/u/무대표
https://maple.gg/u/Q투컴비숍Q
https://maple.gg/u/뽀솜잉
https://maple.gg/u/귤색파르페
https://maple.gg/u/호우그
https://maple.gg/u/정엽옆
https://maple.gg/u/포리교
https://maple.gg/u/간장간장치킨
https://maple.gg/u/계절계절
https://maple.gg/u/천사추v
https://maple.gg/u/무무갬성
https://maple.gg/u/츄신
https://maple.gg/u/배김
https://maple.gg/u/좋은날을
https://maple.gg/u/허디우디
https://maple.gg/u/길티군
https://maple.gg/u/보스는파인더
https://maple.gg/u/고상한여왕님
https:/

In [8]:
flag



'535'

In [36]:
def find_Dojang (name) :
    dojang_floor = 'NON'
    dojang_time = 'NON'
   
    dojang_cur_url = 'https://maple.gg/u/' + name
    
    dojang_cur_res = requests.get(dojang_cur_url)
    dojang_cur_s = BeautifulSoup(dojang_cur_res.content, 'html.parser')

    if dojang_cur_s.find('h1', {'class' : 'user-summary-floor'}) :
        dojang_best_info = dojang_cur_s.find('h1', {'class' : 'user-summary-floor'}).text.replace(' ','').replace('\n','')
        if int(dojang_best_info.replace('층','')) < 10 :
            dojang_best_info = '0' + dojang_best_info
    else:
        dojang_best_info = '00층'

    if(dojang_cur_s.find('tbody')) :
        dojang_latest_info = dojang_cur_s.find('tbody').find('tr').find('h5').text
        if int(dojang_latest_info.replace('층','')) < 10 :
            dojang_latest_info = '0' + dojang_latest_info
    else :
        dojang_latest_info = '00층'
    
    dojang_class_info = dojang_cur_s.find_all('li', {'class' : 'user-summary-item'})[1].text
    
    return [dojang_best_info, dojang_latest_info, dojang_class_info]

find_Dojang ('간장간장치킨')

['48층', '48층', '아란']

In [33]:
floor = '1층'
if int(floor.replace('층','')) < 10 :
    floor = '0' + floor

floor

'01층'

In [4]:
import requests                  # 웹 페이지의 HTML을 가져오는 모듈
from bs4 import BeautifulSoup    # HTML을 파싱하는 모듈
import time
import json
import urllib.request
import pymysql
import os
        
def find_Dojang (name) :
    dojang_floor = 'NON'
    dojang_time = 'NON'
   
    dojang_cur_url = 'https://maple.gg/u/' + name
    
    dojang_cur_res = requests.get(dojang_cur_url)
    dojang_cur_s = BeautifulSoup(dojang_cur_res.content, 'html.parser')

    if dojang_cur_s.find('h1', {'class' : 'user-summary-floor'}) :
        dojang_best_info = dojang_cur_s.find('h1', {'class' : 'user-summary-floor'}).text.replace(' ','').replace('\n','')
        if int(dojang_best_info.replace('층','')) < 10 :
            dojang_best_info = '0' + dojang_best_info
    else:
        dojang_best_info = '00층'

    if(dojang_cur_s.find('tbody')) :
        dojang_latest_info = dojang_cur_s.find('tbody').find('tr').find('h5').text
        if int(dojang_latest_info.replace('층','')) < 10 :
            dojang_latest_info = '0' + dojang_latest_info
    else :
        dojang_latest_info = '00층'
    
    dojang_class_info = dojang_cur_s.find_all('li', {'class' : 'user-summary-item'})[1].text
    
    return [dojang_best_info, dojang_latest_info, dojang_class_info]


In [5]:
find_Dojang('간장간장치킨')

['48층', '48층', '아란']